# Main libraries

In [ ]:
import pandas as pd
import numpy as np

# IPUMS dataset

## reading dataset（https://www.ipums.org/）

In [ ]:
usadf = pd.read_csv('usa_00011.csv')
usadf

In [ ]:
col1 = ['YEAR','MET2013','PROPTX99','COSTELEC','HHINCOME','AGE','BPLD','ANCESTR1','OCC2010',
        'INCEARN','PRESGL','PRENT','PWMET13','UHRSWORK']
cols_to_drop = [col for col in usadf.columns if col not in col1]
usadf = usadf.drop(cols_to_drop, axis=1)
usadf

In [ ]:
usadf = usadf.loc[(usadf['MET2013'] != 0)]
usadf = usadf.loc[(usadf['PROPTX99'] != 0)]
usadf = usadf.loc[(usadf['BPLD'] != 99900)]
usadf = usadf.loc[(usadf['ANCESTR1'] != 999)]
usadf = usadf.loc[(usadf['ANCESTR1'] != 998)]
usadf = usadf.loc[(usadf['ANCESTR1'] != 996)]
usadf = usadf.loc[(usadf['PRENT'] != 0)]
usadf = usadf.loc[(usadf['PWMET13'] != 0)]
usadf = usadf.loc[(usadf['AGE'] >= 16)]
usadf = usadf.loc[(usadf['AGE'] <= 79)]
usadf = usadf.loc[(usadf['YEAR'] > 2019)]
usadf

In [ ]:
print(len(usadf.loc[(usadf['YEAR'] == 2019)]))

In [ ]:
from sklearn.preprocessing import LabelEncoder
col1 = usadf.columns
for i in range(len(col1)):
    val_att = set(usadf[col1[i]])
    print(col1[i],len(set(usadf[col1[i]])))
    if len(set(usadf[col1[i]]))<64:
        usadf = usadf.drop(col1[i], axis=1)
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # lbe = LabelEncoder()
    # usadf[[col1[i]]]=usadf[[col1[i]]].apply(lbe.fit_transform)
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # print(lbe.classes_)

In [ ]:
from sklearn.preprocessing import LabelEncoder
col1 = ['MET2013','PROPTX99','COSTELEC','HHINCOME','AGE','BPLD','ANCESTR1','OCC2010',
        'INCEARN','PRESGL','PRENT','PWMET13','UHRSWORK']
for i in range(len(col1)):
    val_att = set(usadf[col1[i]])
    print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    lbe = LabelEncoder()
    usadf[[col1[i]]]=usadf[[col1[i]]].apply(lbe.fit_transform)
    print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    print(lbe.classes_)

In [ ]:
col1 = ['MET2013','PROPTX99','COSTELEC','HHINCOME','AGE','BPLD','ANCESTR1','OCC2010',
        'INCEARN','PRESGL','PRENT','PWMET13','UHRSWORK']
for i in range(len(col1)):
    block = np.round((max(usadf[col1[i]])-min(usadf[col1[i]]))/64)
    usadf[col1[i]] = (usadf[col1[i]]+abs(min(usadf[col1[i]])))//block
    usadf[col1[i]] = np.clip(usadf[col1[i]], 0, 63)
    usadf[col1[i]] = usadf[col1[i]].astype(int)
    print(max(usadf[col1[i]]),min(usadf[col1[i]]))
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # lbe = LabelEncoder()
    # usadf[[col1[i]]]=usadf[[col1[i]]].apply(lbe.fit_transform)
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # print(lbe.classes_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(usadf['HHWT'], bins = len(x))

In [ ]:
print(usadf.columns)

In [ ]:
col1 = ['MET2013','COSTELEC','AGE','BPLD','ANCESTR1','OCC2010','INCEARN','PRENT','PWMET13','UHRSWORK']
cols_to_drop = [col for col in usadf.columns if col not in col1]
usadf = usadf.drop(cols_to_drop, axis=1)
usadf

In [ ]:
usadf.columns = ['attribute' + str(i) for i in range(len(usadf.columns))]
usadf = usadf.reset_index(drop=True)
usadf

In [ ]:
# 设置随机种子，以确保结果可重复
np.random.seed(42)

# 生成100000行2列的数据
rows = len(usadf)
cols = 4

# 生成属性名
attributes = ['uniform1', 'normal1', 'laplace1', 'prob1']

# 生成数据
uniform_data = np.random.randint(1, 11, size=(rows, 1))
normal_data = np.random.normal(5.5, 2.5, size=(rows, 1))
normal_data = np.clip(normal_data, 1, 10)
laplace_data = np.random.laplace(5, 2, size=(rows, 1))
laplace_data = np.clip(laplace_data, 1, 10)
prob_data = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=(rows, 1), p=[0.18, 0.18, 0.18, 0.0925, 0.0925, 0.0925, 0.0925, 0.03, 0.03, 0.03])

# 将浮点型数据转换为整型
uniform_data = uniform_data.astype(int)
normal_data = normal_data.astype(int)
laplace_data = laplace_data.astype(int)
prob_data = prob_data.astype(int)

# 创建DataFrame
df2 = pd.DataFrame(np.concatenate([uniform_data, normal_data, laplace_data, prob_data], axis=1), columns=attributes)

df2

In [ ]:
merged_df = pd.concat([usadf, df2], axis=1)
merged_df.to_csv('1000000_ipums_64.csv',index=False)

# ACS dataset

In [ ]:
import pandas as pd
import numpy as np
acs_padf = pd.read_csv('ss15pusa.csv')
acs_padf

In [ ]:
acs_pbdf = pd.read_csv('ss15pusb.csv')
acs_pbdf

In [ ]:
acsdf = pd.concat([acs_padf, acs_pbdf], axis=0)

# 如果需要重新设置合并后的索引
acsdf = acsdf.reset_index(drop=True)
acsdf

In [ ]:
col1 = ['ST','AGEP','JWMNP','WKHP','ANC1P','FOD1P','INDP','OCCP','POBP','pwgtp80']
cols_to_drop = [col for col in acsdf.columns if col not in col1]
acsdf = acsdf.drop(cols_to_drop, axis=1)
acsdf

In [ ]:
print(acsdf.columns)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(acsdf['pwgtp80'], bins = len(x))

In [ ]:
acsdf = acsdf.loc[(acsdf['ANC1P'] != 999)]
acsdf = acsdf.loc[(acsdf['ANC1P'] != 998)]
acsdf = acsdf.loc[(acsdf['ANC1P'] != 997)]
acsdf = acsdf.loc[(acsdf['AGEP'] >= 10)]
acsdf = acsdf.loc[(acsdf['AGEP'] <= 73)]
acsdf = acsdf.reset_index(drop=True)
acsdf

In [ ]:
acsdf = acsdf.interpolate(method = 'linear', axis = 0) # 将通过 linear 插值使用同一列的中间值作为填充
acsdf = acsdf.fillna(method = 'backfill', axis = 0) # 将通过前向填充 (ffill) 方法用同一列的后一个数作为填充
acsdf = acsdf.fillna(method = 'ffill', axis = 0) # 将通过 linear 插值使用同一列的中间值作为填充
acsdf

In [ ]:

from sklearn.preprocessing import LabelEncoder
col1 = acsdf.columns
for i in range(len(col1)):
    val_att = set(acsdf[col1[i]])
    print(col1[i], set(acsdf[col1[i]]), len(set(acsdf[col1[i]])))
    lbe = LabelEncoder()
    acsdf[[col1[i]]]=acsdf[[col1[i]]].apply(lbe.fit_transform)
    print(col1[i], set(acsdf[col1[i]]), len(set(acsdf[col1[i]])))
    print(lbe.classes_)

In [ ]:
col1 = acsdf.columns
for i in range(len(col1)):
    block = np.round((max(acsdf[col1[i]])-min(acsdf[col1[i]]))/64)
    acsdf[col1[i]] = (acsdf[col1[i]]+abs(min(acsdf[col1[i]])))//block
    acsdf[col1[i]] = np.clip(acsdf[col1[i]], 0, 63)
    acsdf[col1[i]] = acsdf[col1[i]].astype(int)
    print(max(acsdf[col1[i]]),min(acsdf[col1[i]]))
    # print(col1[i], set(acsdf[col1[i]]), len(set(acsdf[col1[i]])))
    # lbe = LabelEncoder()
    # acsdf[[col1[i]]]=acsdf[[col1[i]]].apply(lbe.fit_transform)
    # print(col1[i], set(acsdf[col1[i]]), len(set(acsdf[col1[i]])))
    # print(lbe.classes_)

In [ ]:
acsdf.columns = ['attribute' + str(i) for i in range(len(acsdf.columns))]
acsdf = acsdf.reset_index(drop=True)
acsdf

In [ ]:
# 设置随机种子，以确保结果可重复
np.random.seed(42)

# 生成100000行2列的数据
rows = len(acsdf)
cols = 4

# 生成属性名
attributes = ['uniform1', 'normal1', 'laplace1', 'prob1']

# 生成数据
uniform_data = np.random.randint(1, 11, size=(rows, 1))
normal_data = np.random.normal(5.5, 2.5, size=(rows, 1))
normal_data = np.clip(normal_data, 1, 10)
laplace_data = np.random.laplace(5, 2, size=(rows, 1))
laplace_data = np.clip(laplace_data, 1, 10)
prob_data = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=(rows, 1), p=[0.18, 0.18, 0.18, 0.0925, 0.0925, 0.0925, 0.0925, 0.03, 0.03, 0.03])

# 将浮点型数据转换为整型
uniform_data = uniform_data.astype(int)
normal_data = normal_data.astype(int)
laplace_data = laplace_data.astype(int)
prob_data = prob_data.astype(int)

# 创建DataFrame
df2 = pd.DataFrame(np.concatenate([uniform_data, normal_data, laplace_data, prob_data], axis=1), columns=attributes)

df2

In [ ]:
merged_df = pd.concat([acsdf, df2], axis=1)
merged_df.to_csv('1000000_acs_64.csv',index=False)

# loan dataset

In [ ]:
import pandas as pd
import numpy as np
loan_adf = pd.read_csv('accepted_2007_to_2018Q4.csv')
loan_adf

In [ ]:
loan_adf = loan_adf.fillna(999999999)
loan_adf

In [ ]:
from sklearn.preprocessing import LabelEncoder
col1 = loan_adf.columns
for i in range(len(col1)):
    val_att = set(loan_adf[col1[i]])
    print(col1[i],len(set(loan_adf[col1[i]])))
    # if len(set(loan_adf[col1[i]]))<64:
    #     loan_adf = loan_adf.drop(col1[i], axis=1)
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # lbe = LabelEncoder()
    # usadf[[col1[i]]]=usadf[[col1[i]]].apply(lbe.fit_transform)
    # print(col1[i], set(usadf[col1[i]]), len(set(usadf[col1[i]])))
    # print(lbe.classes_)

In [ ]:
loan_adf['issue_d'] = loan_adf['issue_d'].replace(999999999, 'z')
loan_adf['zip_code'] = loan_adf['zip_code'].replace(999999999, 'z')
loan_adf['addr_state'] = loan_adf['addr_state'].replace(999999999, 'z')
loan_adf['earliest_cr_line'] = loan_adf['earliest_cr_line'].replace(999999999, 'z')

In [ ]:
from sklearn.preprocessing import LabelEncoder
col1 = ['loan_amnt', 'installment', 'issue_d', 'zip_code', 'addr_state', 'dti', 'earliest_cr_line', 'open_acc', 'avg_cur_bal', 'total_pymnt']
for i in range(len(col1)):
    print(col1[i], set(loan_adf[col1[i]]), len(set(loan_adf[col1[i]])))
    lbe = LabelEncoder()
    loan_adf[[col1[i]]]=loan_adf[[col1[i]]].apply(lbe.fit_transform)
    print(col1[i], set(loan_adf[col1[i]]), len(set(loan_adf[col1[i]])))
    print(lbe.classes_)

In [ ]:
col1 = ['loan_amnt', 'installment', 'issue_d', 'zip_code', 'addr_state', 'dti', 'earliest_cr_line', 'open_acc', 'avg_cur_bal', 'total_pymnt']
cols_to_drop = [col for col in loan_adf.columns if col not in col1]
loan_adf = loan_adf.drop(cols_to_drop, axis=1)
loan_adf

In [ ]:
loan_adf = loan_adf.loc[(loan_adf['loan_amnt'] != max(loan_adf['loan_amnt']))]
loan_adf = loan_adf.loc[(loan_adf['installment'] != max(loan_adf['installment']))]
loan_adf = loan_adf.loc[(loan_adf['issue_d'] != max(loan_adf['issue_d']))]
loan_adf = loan_adf.loc[(loan_adf['zip_code'] != max(loan_adf['zip_code']))]
loan_adf = loan_adf.loc[(loan_adf['addr_state'] != max(loan_adf['addr_state']))]
loan_adf = loan_adf.loc[(loan_adf['dti'] != max(loan_adf['dti']))]
loan_adf = loan_adf.loc[(loan_adf['earliest_cr_line'] != max(loan_adf['earliest_cr_line']))]
loan_adf = loan_adf.loc[(loan_adf['open_acc'] != max(loan_adf['open_acc']))]
loan_adf = loan_adf.loc[(loan_adf['avg_cur_bal'] != max(loan_adf['avg_cur_bal']))]
loan_adf = loan_adf.loc[(loan_adf['total_pymnt'] != max(loan_adf['total_pymnt']))]
loan_adf

In [ ]:
col1 = loan_adf.columns
for i in range(len(col1)):
    block = np.round((max(loan_adf[col1[i]])-min(loan_adf[col1[i]]))/64)
    loan_adf[col1[i]] = (loan_adf[col1[i]]-min(loan_adf[col1[i]]))//block
    loan_adf[col1[i]] = np.clip(loan_adf[col1[i]], 0, 63)
    loan_adf[col1[i]] = loan_adf[col1[i]].astype(int)
    print(max(loan_adf[col1[i]]),min(loan_adf[col1[i]]))
    # print(col1[i], set(loan_adf[col1[i]]), len(set(loan_adf[col1[i]])))
    # lbe = LabelEncoder()
    # loan_adf[[col1[i]]]=loan_adf[[col1[i]]].apply(lbe.fit_transform)
    # print(col1[i], set(loan_adf[col1[i]]), len(set(loan_adf[col1[i]])))
    # print(lbe.classes_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(loan_adf['loan_amnt'], bins = len(x))

In [ ]:
loan_adf.columns = ['attribute' + str(i) for i in range(len(loan_adf.columns))]
loan_adf = loan_adf.reset_index(drop=True)
loan_adf

In [ ]:
# 设置随机种子，以确保结果可重复
np.random.seed(42)

# 生成100000行2列的数据
rows = len(loan_adf)
cols = 4

# 生成属性名
attributes = ['uniform1', 'normal1', 'laplace1', 'prob1']

# 生成数据
uniform_data = np.random.randint(1, 11, size=(rows, 1))
normal_data = np.random.normal(5.5, 2.5, size=(rows, 1))
normal_data = np.clip(normal_data, 1, 10)
laplace_data = np.random.laplace(5, 2, size=(rows, 1))
laplace_data = np.clip(laplace_data, 1, 10)
prob_data = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=(rows, 1), p=[0.18, 0.18, 0.18, 0.0925, 0.0925, 0.0925, 0.0925, 0.03, 0.03, 0.03])

# 将浮点型数据转换为整型
uniform_data = uniform_data.astype(int)
normal_data = normal_data.astype(int)
laplace_data = laplace_data.astype(int)
prob_data = prob_data.astype(int)

# 创建DataFrame
df2 = pd.DataFrame(np.concatenate([uniform_data, normal_data, laplace_data, prob_data], axis=1), columns=attributes)

df2

In [ ]:
merged_df = pd.concat([loan_adf, df2], axis=1)
merged_df.to_csv('1000000_loan_64.csv',index=False)

In [ ]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0, 64)
xU, xL = x + 0.5, x - 0.5 
prob = ss.norm.cdf(xU, scale = 1) - ss.norm.cdf(xL, scale = 1)
prob = prob / prob.sum() # normalize the probabilities so their sum is 1
nums = np.random.choice(x, size = 10000, p = prob)
plt.hist(nums, bins = len(x))

In [ ]:
nums = nums+5
print(set(nums))

In [ ]:
import numpy
x = np.arange(1, 11)
x2 = numpy.random.randint(low=1, high=11, size=45223, dtype='l')
print(set(x2), len(set(x2)))
plt.hist(x2, bins = len(x))

In [ ]:

df_epsilon = pd.DataFrame()
df_epsilon['uniform1'] = x2
df_epsilon['normal1'] = nums
df_epsilon.to_csv('db_adults_epsilon.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
# 设置随机种子，以确保结果可重复
np.random.seed(42)
# 生成100000行10列的数据
rows = 1000000
cols = 10

# 生成属性名
attributes = ['attribute' + str(i) for i in range(cols)]

# 生成数据
# data = np.random.uniform(0, 64, size=(rows, cols))
# data = np.random.normal(loc=31.5, scale=15, size=(rows, cols))
# data = np.clip(data, 0, 63)
data = np.random.laplace(loc=0, scale=1, size=(rows, cols))
data_min = np.min(data)
data_max = np.max(data)
data = (data - data_min) / (data_max - data_min) * 63
data = np.clip(data, 0, 63)
data = data.astype(int)

# 创建DataFrame
df1 = pd.DataFrame(data, columns=attributes)

# 打印DataFrame
print(df1)

In [ ]:
df1.to_csv('uniform.csv',index=False)

In [ ]:
# 设置随机种子，以确保结果可重复
np.random.seed(42)

# 生成100000行2列的数据
rows = 1000000
cols = 4

# 生成属性名
attributes = ['uniform1', 'normal1', 'laplace1', 'prob1']

# 生成数据
uniform_data = np.random.randint(1, 11, size=(rows, 1))
normal_data = np.random.normal(5.5, 2.5, size=(rows, 1))
normal_data = np.clip(normal_data, 1, 10)
laplace_data = np.random.laplace(5, 2, size=(rows, 1))
laplace_data = np.clip(laplace_data, 1, 10)
prob_data = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=(rows, 1), p=[0.18, 0.18, 0.18, 0.0925, 0.0925, 0.0925, 0.0925, 0.03, 0.03, 0.03])

# 将浮点型数据转换为整型
uniform_data = uniform_data.astype(int)
normal_data = normal_data.astype(int)
laplace_data = laplace_data.astype(int)
prob_data = prob_data.astype(int)

# 创建DataFrame
df2 = pd.DataFrame(np.concatenate([uniform_data, normal_data, laplace_data, prob_data], axis=1), columns=attributes)

df2

In [ ]:
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of samples and the number of attributes
num_samples = 1000000
num_attributes = 10

# Generate random covariance matrix
covariance_matrix = np.random.rand(num_attributes, num_attributes)

# Set half of the covariances to be less than 0.3
covariance_matrix[covariance_matrix < 0.6] = np.random.uniform(0, 0.2, size=np.sum(covariance_matrix < 0.6))

# Set remaining covariances to be greater than 0.3
covariance_matrix[covariance_matrix >= 0.6] = np.random.uniform(0.2, 0.8, size=np.sum(covariance_matrix >= 0.6))

# Make the covariance matrix symmetric
covariance_matrix = np.triu(covariance_matrix) + np.triu(covariance_matrix, 1).T

# 计算特征值分解
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# 将负特征值设置为零
eigenvalues[eigenvalues < 0] = 0

# 重新构造对称半正定矩阵
covariance_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T))
np.fill_diagonal(covariance_matrix, 1)

# Generate the dataset
data = np.random.multivariate_normal(mean=[0] * num_attributes, cov=covariance_matrix, size=num_samples)

# Scale the data to the range of 0-63
data_min = np.min(data)
data_max = np.max(data)
scaled_data = (data - data_min) / (data_max - data_min) * 63

# Round the scaled data to the nearest integer
rounded_data = np.round(scaled_data)
rounded_data = rounded_data.astype(int)
print(np.max(rounded_data))
print(np.min(rounded_data))

In [ ]:
import numpy as np

np.random.seed(42)

num_samples = 1000000
num_attributes = 10

# Generate random covariance matrix
cov = np.random.rand(num_attributes, num_attributes)

# Set half of the covariances to be less than 0.3
cov[cov < 0.6] = np.random.uniform(0, 0.2, size=np.sum(cov < 0.6))

# Set remaining covariances to be greater than 0.3
cov[cov >= 0.6] = np.random.uniform(0.2, 0.8, size=np.sum(cov >= 0.6))

# Make the covariance matrix symmetric
cov = np.triu(cov) + np.triu(cov, 1).T
# 计算特征值分解
eigenvalues, eigenvectors = np.linalg.eig(cov)

# 将负特征值设置为零
eigenvalues[eigenvalues < 0] = 0

# 重新构造对称半正定矩阵
cov = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T))
np.fill_diagonal(cov, 1)
# Generate samples from the multivariate uniform distribution
data = np.random.uniform(low=0, high=1, size=(num_samples, num_attributes))

# Apply the covariance matrix to the data
data = np.dot(data, np.linalg.cholesky(cov).T)

# Scale the data to the range of 0-63
data_min = np.min(data)
data_max = np.max(data)
scaled_data = (data - data_min) / (data_max - data_min) * 63

# Round the scaled data to the nearest integer
rounded_data = np.round(scaled_data)
rounded_data = rounded_data.astype(int)
print(np.max(rounded_data))
print(np.min(rounded_data))

In [ ]:
import numpy as np

np.random.seed(42)

num_samples = 1000000
num_attributes = 10

# Generate random covariance matrix
cov = np.random.rand(num_attributes, num_attributes)

# Set half of the covariances to be less than 0.3
cov[cov < 0.6] = np.random.uniform(0, 0.2, size=np.sum(cov < 0.6))

# Set remaining covariances to be greater than 0.3
cov[cov >= 0.6] = np.random.uniform(0.2, 0.8, size=np.sum(cov >= 0.6))

# Make the covariance matrix symmetric
cov = np.triu(cov) + np.triu(cov, 1).T
# 计算特征值分解
eigenvalues, eigenvectors = np.linalg.eig(cov)

# 将负特征值设置为零
eigenvalues[eigenvalues < 0] = 0

# 重新构造对称半正定矩阵
cov = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T))
np.fill_diagonal(cov, 1)
# Generate samples from the multivariate uniform distribution
data = np.random.laplace(loc=0, scale=1, size=(num_samples, num_attributes))

# Apply the covariance matrix to the data
data = np.dot(data, np.linalg.cholesky(cov).T)

# Scale the data to the range of 0-63
data_min = np.min(data)
data_max = np.max(data)
scaled_data = (data - data_min) / (data_max - data_min) * 63

# Round the scaled data to the nearest integer
rounded_data = np.round(scaled_data)
rounded_data = rounded_data.astype(int)
print(np.max(rounded_data))
print(np.min(rounded_data))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(rounded_data, bins = len(x))

In [ ]:
attributes = ['attribute' + str(i) for i in range(10)]
df1 = pd.DataFrame(rounded_data, columns=attributes)

In [ ]:
# 保存为CSV文件
df2.to_csv('uniform_epsilon.csv', index=False)

In [ ]:

# 合并两个数据框
merged_df = pd.concat([df1, df2], axis=1)

# # 按指定列和Grouper进行划分
# grouped_df = merged_df.groupby('laplace1')

# # 保存划分后的结果为CSV文件
# grouped_df = grouped_df.apply(lambda merged_df: merged_df.sort_values(by=['laplace1']).reset_index(drop=True))
merged_df.to_csv('1000000_laplace_64_1.csv')

In [ ]:
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of samples and the number of attributes
num_samples = 1000000
num_attributes = 10

# Generate random covariance matrix
covariance_matrix = np.random.rand(num_attributes, num_attributes)

# Set half of the covariances to be less than 0.3
covariance_matrix[covariance_matrix < 0.6] = np.random.uniform(0, 0.2, size=np.sum(covariance_matrix < 0.6))

# Set remaining covariances to be greater than 0.3
covariance_matrix[covariance_matrix >= 0.6] = np.random.uniform(0.2, 0.8, size=np.sum(covariance_matrix >= 0.6))

# Make the covariance matrix symmetric
covariance_matrix = np.triu(covariance_matrix) + np.triu(covariance_matrix, 1).T
# Fill the diagonal with 1's
# 计算特征值分解
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# 将负特征值设置为零
eigenvalues[eigenvalues < 0] = 0

# 重新构造对称半正定矩阵
covariance_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T))
np.fill_diagonal(covariance_matrix, 1)
print(covariance_matrix)
# Generate the dataset
data = np.random.multivariate_normal(mean=[0] * num_attributes, cov=covariance_matrix, size=num_samples)


In [ ]:
import numpy as np
import pandas as pd
# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of samples and the number of attributes
num_samples = 10000000
num_attributes = 10

# Generate random samples from a multivariate distribution
# data = np.random.normal(loc=0, scale=1, size=(num_samples, num_attributes))
data = np.random.laplace(loc=0, scale=1, size=(num_samples, num_attributes))
# data = np.random.uniform(low=0, high=1, size=(num_samples, num_attributes))
# Scale the data to the range of 0-63
data_min = np.min(data)
data_max = np.max(data)
scaled_data = (data - data_min) / (data_max - data_min) * 63

# Round the scaled data to the nearest integer
rounded_data = np.round(scaled_data)
rounded_data = rounded_data.astype(int)
print(np.max(rounded_data))
print(np.min(rounded_data))
rounded_data[np.isnan(rounded_data)] = 0
attributes = ['attribute' + str(i) for i in range(10)]
df1 = pd.DataFrame(rounded_data, columns=attributes)

df1

In [ ]:
# 设置随机种子，以确保结果可重复
np.random.seed(42)

# 生成100000行2列的数据
rows = len(df1)
cols = 4

# 生成属性名
attributes = ['uniform1', 'normal1', 'laplace1', 'prob1']

# 生成数据
uniform_data = np.random.randint(1, 11, size=(rows, 1))
normal_data = np.random.normal(5.5, 2.5, size=(rows, 1))
normal_data = np.clip(normal_data, 1, 10)
laplace_data = np.random.laplace(5, 2, size=(rows, 1))
laplace_data = np.clip(laplace_data, 1, 10)
prob_data = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=(rows, 1), p=[0.18, 0.18, 0.18, 0.0925, 0.0925, 0.0925, 0.0925, 0.03, 0.03, 0.03])

# 将浮点型数据转换为整型
uniform_data = uniform_data.astype(int)
normal_data = normal_data.astype(int)
laplace_data = laplace_data.astype(int)
prob_data = prob_data.astype(int)

# 创建DataFrame
df2 = pd.DataFrame(np.concatenate([uniform_data, normal_data, laplace_data, prob_data], axis=1), columns=attributes)

df2

In [ ]:

# 合并两个数据框
merged_df = pd.concat([df1, df2], axis=1)

# # 按指定列和Grouper进行划分
# grouped_df = merged_df.groupby('laplace1')

# # 保存划分后的结果为CSV文件
# grouped_df = grouped_df.apply(lambda merged_df: merged_df.sort_values(by=['laplace1']).reset_index(drop=True))
merged_df.to_csv('10000000_laplace_64_.csv')

In [ ]:
pd.read_csv('usa_00010.csv')

In [ ]:
ipumsdf = pd.read_csv('1000000_ipums_64.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
x = np.arange(64)
plt.hist(ipumsdf['attribute8'], bins = len(x))